In [4]:
import numpy as np
import pandas as pd
import datatable as dt

DATA_PATH = '../riiid-test-answer-prediction'

In [5]:
import datatable as dt
#train = dt.fread(f"{DATA_PATH}/train.csv")
lectures = dt.fread(f"{DATA_PATH}/lectures.csv")
questions = dt.fread(f"{DATA_PATH}/questions.csv")
#train = train.to_pandas()
lectures = lectures.to_pandas()
questions = questions.to_pandas()


In [6]:
train = pd.read_csv(
    f'{DATA_PATH}/trn_eda.csv', 
    dtype={
        'row_id': 'int64', 
        'timestamp': 'int64', 
        'user_id': 'int32', 
        'content_id': 'int16', 
        'content_type_id': 'int8',
        'task_container_id': 'int16', 
        'user_answer': 'int8', 
        'answered_correctly': 'int8', 
        'prior_question_elapsed_time': 'float32', 
        'prior_question_had_explanation': 'boolean'
    }
)

train.head()


,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,9023,0,81429,7900,0,0,0,1,NaN,<NA>
1,9024,19803,81429,7876,0,1,3,1,8000.0,False
2,9025,40823,81429,175,0,2,1,0,18000.0,False
3,9026,57316,81429,1278,0,3,3,1,19000.0,False
4,9027,111785,81429,2065,0,4,0,0,15000.0,False


In [7]:
train.describe()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time
count,7.185049e+06,7.185049e+06,7.185049e+06,7.185049e+06,7.185049e+06,7.185049e+06,7.185049e+06,7.185049e+06,7.018059e+06
mean,4.994508e+07,7.668560e+09,1.062535e+09,5.204463e+03,1.941726e-02,9.064258e+02,1.375707e+00,6.243802e-01,2.546931e+04
std,2.911799e+07,1.174988e+10,6.174980e+08,3.874040e+03,1.379864e-01,1.349693e+03,1.193220e+00,5.228424e-01,2.033400e+04
min,9.023000e+03,0.000000e+00,8.142900e+04,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00
25%,2.462121e+07,5.251792e+08,5.268930e+08,2.050000e+03,0.000000e+00,1.060000e+02,0.000000e+00,0.000000e+00,1.600000e+04
50%,4.948124e+07,2.611608e+09,1.048634e+09,4.998000e+03,0.000000e+00,3.860000e+02,1.000000e+00,1.000000e+00,2.100000e+04
75%,7.470872e+07,9.511717e+09,1.589246e+09,7.402000e+03,0.000000e+00,1.102000e+03,3.000000e+00,1.000000e+00,2.966600e+04
max,1.012287e+08,8.405387e+10,2.147463e+09,3.273600e+04,1.000000e+00,9.999000e+03,3.000000e+00,1.000000e+00,3.000000e+05


In [8]:
train.shape

(7185049, 10)

In [9]:

unique_tags = set().union(*[y.split() for y in questions['tags'].astype(str).values])
print(f"There are {len(unique_tags)} different tags")


There are 188 different tags


In [1]:
# Cut training set further:


### Joining main data with questions data

In [10]:
lectures_joined = train[train.content_type_id==1].merge(lectures, left_on='content_id', right_on='lecture_id')
lectures_joined.drop(columns='lecture_id', inplace=True)
questions_joined = train[train.content_type_id==0].merge(questions, left_on='content_id', right_on='question_id')
questions_joined.drop(columns='question_id', inplace=True)
print(train.shape, lectures_joined.shape, questions_joined.shape, lectures_joined.shape[0] + questions_joined.shape[0])



(7185049, 10) (139514, 13) (7045535, 14) 7185049


In [11]:
lectures_joined.answered_correctly.unique()

array([-1], dtype=int8)

We see that the answered_correctly are always -1, so we only need to work on a model where the content type is a question. 

### Joining main data including questions data with lectures data

In [43]:
# task container ID:
set(questions_joined.task_container_id).intersection(set(lectures_joined.task_container_id))

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [57]:
t = questions_joined[questions_joined.task_container_id == 0]
t

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,bundle_id,correct_answer,part,tags
0,9023,0,81429,7900,0,0,0,1,NaN,<NA>,7900,0,1,131 93 81
1,14162,0,140969,7900,0,0,0,1,NaN,<NA>,7900,0,1,131 93 81
2,16528,0,146023,7900,0,0,0,1,NaN,<NA>,7900,0,1,131 93 81
3,22919,0,384661,7900,0,0,1,0,NaN,<NA>,7900,0,1,131 93 81
4,34121,0,900423,7900,0,0,2,0,NaN,<NA>,7900,0,1,131 93 81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7039856,96502217,0,2047724707,6350,0,0,2,1,NaN,<NA>,6350,2,5,79
7039859,99883423,0,2120132918,6350,0,0,2,1,NaN,<NA>,6350,2,5,79
7041081,29903281,0,640773107,4621,0,0,0,1,NaN,<NA>,4621,0,5,79
7041137,68753832,0,1462814260,4621,0,0,0,1,NaN,<NA>,4621,0,5,79


In [44]:
lectures_joined[lectures_joined.task_container_id == 0]

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,tag,part,type_of
22568,72169013,0,1536583938,5694,1,0,-1,-1,NaN,False,140,2,concept
77679,2955157,228551,64241092,32312,1,0,-1,-1,NaN,False,176,2,concept
101524,81943738,0,1743991910,18186,1,0,-1,-1,NaN,False,161,4,concept
136227,69588711,0,1480072126,31077,1,0,-1,-1,NaN,False,130,1,concept


we observe that task container ID between questions and lectures have no common users. so pretty much useless column

Joining Algorithm:

For each question, find the time/type of last lecture attended by the same user of that same tag.

In [16]:
# DONT RUN THE FOLLOWING:

In [ ]:
lectures_joined = lectures_joined.sort_values('timestamp', ascending=False)

last_lecture_timestamps = []
last_lecture_types = []
for _, q in questions_joined.iterrows():
    last_lecture_timestamp = -1
    last_lecture_type = ""
    for _, l in lectures_joined.iterrows():
        if l.timestamp > q.timestamp:
            continue
        
        if l.user_id == q.user_id and l.part == q.part and l.tag in [int(t) for t in q.tags.split()]:
            last_lecture_timestamp = l.timestamp
            last_lecture_type = l.lecture_type_of
            break
    last_lecture_timestamps.append(last_lecture_timestamp)
    last_lecture_types.append(last_lecture_type)

In [ ]:
questions_joined['prior_lecture_timestamp'] = last_lecture_timestamps
questions_joined['prior_lecture_type'] = last_lecture_types

In [ ]:
# Takes too long. Going to ignore lecture data for now

### Exploring joined questions dataset

In [12]:
import plotly.express as px


In [13]:
# rate of correct answers:
ds = questions_joined['answered_correctly'].value_counts().reset_index()
ds.columns = ['answered_correctly', 'percent_of_answers']
ds['percent_of_answers'] /= len(questions_joined)
ds = ds.sort_values(['percent_of_answers'])

fig = px.pie(
    ds, 
    names='answered_correctly', 
    values='percent_of_answers', 
    title='Percent of correct answers', 
    width=800,
    height=500 
)

#fig.show() #TAKES UP TOO MUCH MEM


Seems to be more answered correctly than not, which makes sense

In [14]:
# correct answers for every option:
ds = questions['correct_answer'].value_counts().reset_index()
ds.columns = ['correct_answer', 'number_of_answers']
ds['correct_answer'] = ds['correct_answer'].astype(str) + '-'
ds = ds.sort_values(['number_of_answers'])

fig = px.bar(
    ds, 
    x='number_of_answers', 
    y='correct_answer', 
    orientation='h', 
    title='Number of correct answers per group', 
    width=800,
    height=300
)

#fig.show() #TAKES UP TOO MUCH MEM




seems fairly even, no answer group is clearly dominating

In [15]:
# explore using prior question elapsed time
# prior question elapsed time distribution:

fig = px.histogram(
    questions_joined, 
    x="prior_question_elapsed_time",
    nbins=50,
    title='prior_question_elapsed_time distribution',
    width=800,
    height=500
)

#fig.show()


Most prior questions were answered within the last 50 seconds

### Exploring relationships

In [16]:
# relationship between prior question elapsed time and answered correctly

#fig = px.scatter(
#    questions_joined, 
#    x="prior_question_elapsed_time",
#    y="answered_correctly",
#    title='answered_correctly vs prior_question_elapsed_time',
#    width=800,
#    height=500
#)

#fig.show()


Can't see much, will look at distribution in answered correctly vs not

In [17]:
# prior question elapsed time of those answered correctly

fig = px.histogram(
    questions_joined[questions_joined.answered_correctly == 1], 
    x="prior_question_elapsed_time",
    nbins=50,
    title='prior_question_elapsed_time of those answered correctly',
    width=800,
    height=500
)

#fig.show()


In [18]:
# prior question elapsed time of those answered incorrectly

fig = px.histogram(
    questions_joined[questions_joined.answered_correctly == 0], 
    x="prior_question_elapsed_time",
    nbins=50,
    title='prior_question_elapsed_time of those answered incorrectly',
    width=800,
    height=500
)

#fig.show()
